In [7]:
# Remember : We're building a character level language model !!!

In [1]:
# Downloading the Tiny Shakespeare Dataset

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-01-19 10:04:01--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.08MB/s    in 0.3s    

2023-01-19 10:04:02 (3.08 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# Reading the data

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("Length of Dataset in characters : ", len(text))

Length of Dataset in characters :  1115394


In [5]:
# Sample data
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# Unique characters that occur in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [18]:
# Create a mapping from characters to integers
stoi = {ch : i for i,ch in enumerate(chars)}
itos = {i : ch for i,ch in enumerate(chars)}

# Encode takes in a string and returns a list of integers
# Decode takes in a list of integers and returns a string

encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

# Side Note

# Google uses SentencePiece tokenizer, which is on a sub-word level
# OpenAI used tiktoken library's Byte Pair Encoding (BPE) for GPT-2

In [21]:
# Example

# print(encode("hii there"))
# print(decode(encode("hii there")))

In [22]:
# We'll encode the entire dataset and store it in a torch.tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [23]:
# Splitting the data into train and validation sets

n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

In [24]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [25]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target : {target}")

when input is tensor([18]) the target : 47
when input is tensor([18, 47]) the target : 56
when input is tensor([18, 47, 56]) the target : 57
when input is tensor([18, 47, 56, 57]) the target : 58
when input is tensor([18, 47, 56, 57, 58]) the target : 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target : 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target : 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target : 58


In [27]:
# Making batches

torch.manual_seed(1337)

# How many independent sequences will we process in parallel?
batch_size = 4

# What is the maximum context length for predictions?
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
#     print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')

print("inputs: ")
print(xb.shape)
print(xb)

print("targets: ")
print(yb.shape)
print(yb)

print('---------')

for b in range(batch_size):     # Batch Dimension 
    for t in range(block_size): # Time  Dimension
        context = xb[b, :t+1]
        target  = yb[b, t]
        print(f"when input is {context} the target : {target}")

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------
when input is tensor([24]) the target : 43
when input is tensor([24, 43]) the target : 58
when input is tensor([24, 43, 58]) the target : 5
when input is tensor([24, 43, 58,  5]) the target : 57
when input is tensor([24, 43, 58,  5, 57]) the target : 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target : 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target : 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target : 39
when input is tensor([44]) the target : 53
when input is tensor([44, 53]) the target : 56
when input is tensor([44, 53, 56]) the target : 1
w

In [28]:
# Input to the transformer
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [38]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        
#       each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets = None):
              
#              idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B, T, C)

        if (targets == None):
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T, )
            loss = F.cross_entropy(logits, targets)
      
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
#         idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
#           get predictions
            logits, loss = self(idx) 
            
#           focus only on the last timestep
            logits = logits[:, -1, :] # becomes (B, C)

#           apply softmax to get probabilities     
            probs = F.softmax(logits, dim = -1) # (B, C)
    
#           sample from the distribution to generate next character
            idx_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
    
#           append sampled index to the running index
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T + 1)
    
        return idx
    
    
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


print(decode(m.generate(torch.zeros((1, 1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [40]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [41]:
# training loop

batch_size = 32
for steps in range(10000):
    
#     sample a batch of data
    xb, yb = get_batch('train')
    
#     evaluate loss
    logits, loss = m(xb, yb)
    
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.382369041442871


In [44]:
print(decode(m.generate(torch.zeros((1, 1), dtype = torch.long), max_new_tokens = 500)[0].tolist()))



PRO f athawendesees yof th withS:

FiFLINR:

Wheader y blitow,
Ye m o ditoshyd me, ch rte u hart ararwsa
Wou fe,
INurathoune
IESSARin,
MIOLened sus;
Wh.
S:
NMy BAnind g.
iudshank
An chin is a arokisupxaseru t w ity merwo al LOLo bebte loolld worinero ya l aknge ond thal ttry b's mo ge ck.

gh, inketilllin trk$nutud t ar,
WAnt cithap's Zimponcrdistherdrtes saure ' erpoperrposthel?
Handis of hef thep: ct
Ywit harfoul'st, ar izlor t ct.
Fo, sther:
I d tre th,-she.
Wowltothedl:
NNONANRI, aft,
STo w


In [45]:
# Mathematical Trick for Self-Attention

In [51]:
# Consider the following toy example

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [52]:
# We want x[b, t] = mean_{i<=t} x[b,i]

xbow = torch.zeros((B, T, C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (T, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [70]:
# v2 : Making the code efficient by replicating the above as matrix mult

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim = True)
xbow2 = wei @ x  # (T, T) @ (B, T, C) ----> (B, T, T) @ (B, T, C) ----> (B, T, C) 
torch.allclose(xbow, xbow2)

False

In [71]:
xbow.shape, xbow2.shape

(torch.Size([4, 8, 2]), torch.Size([4, 8, 2]))

In [69]:
# xbow[0], xbow2[0]

In [75]:
# v3 : use Softmax

tril = torch.tril(torch.ones(T, T)) 
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x  # (T, T) @ (B, T, C) ----> (B, T, T) @ (B, T, C) ----> (B, T, C) 
torch.allclose(xbow2, xbow3)

True

In [80]:
# v4 : Self Attention
torch.manual_seed(1337)

B, T, C = 4, 8, 32
x = torch.randn(B, T, C)


# Let's see a single head perform self-attention

head_size = 16

key   = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)     # (B, T, 16)
q = query(x)   # (B, T, 16)
v = value(x)   # (B, T, 16)

wei  = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T) 
tril = torch.tril(torch.ones(T, T))
wei  = wei.masked_fill(tril == 0, float('-inf'))
wei  = F.softmax(wei, dim = -1)
out  = wei @ v  # (B, T, T) @ (B, T, head_size) ----> (B, T, T) @ (B, T, head_size) ----> (B, T, head_size) 

out.shape

torch.Size([4, 8, 16])

In [79]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)